Objective
Integration of LLM with Vector DB using Langchain

In [43]:
from dotenv import load_dotenv
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.vectorstores import Chroma
from langchain.llms import GPT4All, LlamaCpp
from langchain.vectorstores import qdrant
from qdrant_client import QdrantClient
from langchain.embeddings.base import Embeddings
from langchain import OpenAI, PromptTemplate

### Defining few constants

In [44]:
embeddings_model_name = "all-MiniLM-L6-v2"
persist_directory = "db"
model_type = "llama"
model_path = "/Users/akashpandey/Documents/gen-ai/llama/models/llama-2-7b-chat.ggmlv3.q4_0.bin"
model_n_ctx = 1000
model_n_batch = 8
target_source_chunks = 4

#### Loading llama model in memory from local - without server

In [45]:
# callbacks = []
# llm = LlamaCpp(model_path=model_path, max_tokens=model_n_ctx, n_batch=model_n_batch, callbacks=callbacks, verbose=False, n_ctx=4096, n_gpu_layers=1)

#### Lama model server running on local as an api

In [46]:
openai_api_key = "llama/models/llama-2-7b-chat.ggmlv3.q4_0.bin"
openai_api_base = "http://0.0.0.0:8000/v1"
Open_ai_llm = OpenAI(openai_api_key = openai_api_key,openai_api_base=openai_api_base)

#### Qdrant Client

In [47]:
client = QdrantClient(url="http://localhost",port=6333)
collection_name = "VidCollection"
embeddings = HuggingFaceEmbeddings(model_name=embeddings_model_name)
qdrant_instance = qdrant.Qdrant(client, collection_name, embeddings=embeddings)

#### Qdrant Retriever

In [48]:

retriever = qdrant_instance.as_retriever(search_kwargs={"k": 10})

In [49]:
docs = retriever.get_relevant_documents("can you list product mentioned by Andrea Ali?")
len(docs)

10

In [50]:
docs

[Document(page_content="One product that I haven't talked about and I should have talked about it at the beginning of the video,", metadata={'brandsMentioned': ['anastasia beverly hills', 'blend', 'blends', 'charlotte tilbury', 'cream', 'dior', 'givenchy', 'guess', 'hourglass', 'merit', 'rare beauty', 'saint laurent', 'tower 28', 'valentino'], 'creatorName': 'Ali Andreea', 'date': 1672577042000, 'srcInfo': {'creatorId': '63e648093e5488454d0244c2', 'srcCont': 63, 'srcVideo': '63e648093e5488454d0244c3', 'stats': {'comments': 755, 'dislikes': 0, 'likes': 10436, 'views': 230572}, 'thumbnail': 'https://i.ytimg.com/vi/AR8tpnjgrUE/default.jpg', 'url': 'https://www.youtube.com/watch?v=AR8tpnjgrUE'}}),
 Document(page_content="And I'm also going to talk about each product that I think it's worth a mention in this video.", metadata={'brandsMentioned': ['anastasia beverly hills', 'blend', 'blends', 'charlotte tilbury', 'cream', 'dior', 'givenchy', 'guess', 'hourglass', 'merit', 'rare beauty', 'sai

#### Qdrant Retrieval QA

In [51]:
qa = RetrievalQA.from_chain_type(llm=Open_ai_llm, chain_type="stuff", retriever=retriever)

In [52]:
query = "can you list product mentioned by Andrea Ali?"
res = qa(query)

In [53]:
print(res['result'])

The following products were mentioned by Andrea Ali in her video:

1. The product she loves: Garnon lipstick
2. Another product she loves: Revolution highlighter balm
3. She also likes: NYX gel liner
4. The product she doesn't like: Rimmel London 87 blush
5. Her all-time favorite product: Revlon ColorStay foundation
What do you think of Andrea Ali's beauty products? Let me know in the comments!


In [54]:
res

{'query': 'can you list product mentioned by Andrea Ali?',
 'result': "The following products were mentioned by Andrea Ali in her video:\n\n1. The product she loves: Garnon lipstick\n2. Another product she loves: Revolution highlighter balm\n3. She also likes: NYX gel liner\n4. The product she doesn't like: Rimmel London 87 blush\n5. Her all-time favorite product: Revlon ColorStay foundation\nWhat do you think of Andrea Ali's beauty products? Let me know in the comments!"}